In [254]:
import requests
import numpy as np
import pandas as pd
import herepy
import config
import json
import isodate
from datetime import datetime as dt

In [286]:
residents = pd.read_csv("addresses_simulated_incomes.csv")
pittsburgh_mask = residents['MUNICIPALITY']=='PITTSBURGH'
pittsburgh = residents[pittsburgh_mask]
car_mask = pittsburgh['simulated_commute_car']==1
residents_car = pittsburgh[car_mask].iloc[0:50]
residents_transit = pittsburgh[-car_mask].iloc[0:50]
#Need to add store and vacant lot locations into a single df. Idk where current store info is--can you add that and the
#vacant lot data here?
lots = pd.read_csv("vacant_lots_centroid.csv")
stores = pd.read_csv("supermarkets.csv")

(61, 11)

In [287]:
#limited rows for testing. Change to residents_car.loc[:, [..]].values instead of the slices to use whole datasets
start_car_coords = residents_car.loc[:,["LATITUDE", "LONGITUDE"]].values
start_transit_coords = residents_transit.loc[:,["LATITUDE", "LONGITUDE"]].values
end_coords_lots = lots.loc[:,["Y", "X"]].values
end_coords_stores = stores.loc[:,["Latitude","Longitude"]].values
end_coords = np.concatenate((end_coords_lots,end_coords_stores),axis=0)[0:100]

routingApi = herepy.RoutingApi(config.api_key)
publicTransitApi = herepy.PublicTransitApi(config.api_key)

In [294]:
# create blank lists that will be populated with trip characteristics
car_origin= []
car_destination= []
car_trip_miles= []
car_trip_time= []

#use car_api_start and car_api_end to time how long the API takes to run
car_api_start = dt.now()

#api is breaking without indicating the problematic row, so need to for loop it and wrap in try except
for i in range(0,len(start_car_coords)):
    for j in range(0,len(end_coords)):
        car_origin.append(i)
        car_destination.append(j)
        try:
            response_car = routingApi.matrix(
                    start_waypoints=start_car_coords[i:(i+1)],
                    destination_waypoints=end_coords[j:(j+1)],
                    departure='2020-08-04T17:00:00+02',
                    summary_attributes = ['traveltime','distance'],
                    modes=[herepy.RouteMode.fastest, herepy.RouteMode.car])
            #extract trip from response
            trip = response_car.as_dict()['response']['matrixEntry'][0]
            # store values
            car_trip_miles.append(trip['summary']['distance']/1609.34)
            car_trip_time.append(trip['summary']['travelTime']/3600)
        except:
            car_trip_miles.append(9999999)
            car_trip_time.append(999999999)


car_api_end = dt.now()


In [296]:

car_results = pd.DataFrame({"origin":car_origin,
              "destination":car_destination,
              "trip_distance":car_trip_miles,
              "trip_time":car_trip_time})
car_results['travel_cost'] = (car_results['trip_distance']*.575) + car_results['trip_time']*7.5

#store costs in i by k matrix
car_time_mat = car_results.pivot(index='origin',columns='destination',values='trip_time')
car_cost_mat = car_results.pivot(index='origin',columns='destination',values='travel_cost')

car_time_mat

destination,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
origin,,,,,,,,,,,,,,,,,,,,,
0,0.283889,0.330833,0.357222,0.141389,0.068611,0.363611,0.314722,0.095556,0.270278,0.324167,...,0.403611,0.456944,0.367222,0.254722,0.309722,0.381667,0.316667,0.348889,0.188056,0.294722
1,0.506111,0.214167,0.205278,0.346111,0.355556,0.174722,0.536944,0.245000,0.181944,0.305833,...,0.105833,0.438611,0.250278,0.476944,0.191389,0.130278,0.180000,0.166667,0.471111,0.276389
2,0.264167,0.308056,0.334444,0.391389,0.276389,0.340833,0.295000,0.288611,0.247500,0.085278,...,0.369722,0.295278,0.312500,0.235000,0.286944,0.358889,0.293889,0.326111,0.320556,0.041944
3,0.417500,0.278889,0.305278,0.226667,0.236111,0.311667,0.448333,0.125556,0.218333,0.298611,...,0.331667,0.431389,0.283333,0.388333,0.257778,0.328611,0.264722,0.288889,0.351667,0.269167
4,0.387500,0.340556,0.374167,0.418889,0.293056,0.373333,0.419444,0.317778,0.280000,0.133056,...,0.394722,0.273333,0.337500,0.358333,0.311944,0.383889,0.315278,0.351111,0.443889,0.136389
5,0.508611,0.376667,0.333611,0.476667,0.414167,0.412500,0.540556,0.375556,0.319167,0.217500,...,0.473333,0.217778,0.340000,0.479444,0.340556,0.430556,0.338333,0.397778,0.565000,0.257500
6,0.264167,0.308056,0.334444,0.391389,0.276389,0.340833,0.295000,0.288611,0.247500,0.085278,...,0.369722,0.295278,0.312500,0.235000,0.286944,0.358889,0.293889,0.326111,0.320556,0.041944
7,0.416944,0.250833,0.277222,0.248056,0.235000,0.283611,0.447778,0.146944,0.190278,0.272222,...,0.323056,0.405000,0.283333,0.387778,0.229722,0.301667,0.236667,0.268889,0.373056,0.242778
8,0.429167,0.268056,0.202222,0.349444,0.326389,0.295000,0.460000,0.248333,0.201667,0.286944,...,0.355833,0.347500,0.208611,0.400000,0.231944,0.313056,0.220833,0.280278,0.474444,0.263611


In [297]:
#it took 8 and a half minutes to run 50 starts by 100 destinations, doing the starts in batches of 15
#should speed things up
(car_api_end-car_api_start).seconds/60

8.6

In [298]:
#try to use the public transit api for transit starting points, and if it fails
#because there is no public transit stop nearby, use walking directions
#create a blank list to store transit_times
transit_origins = []
transit_destinations = []
transit_times = []
transit_api_start = dt.now()
transit_fares = []
for i in range(0,len(start_transit_coords)):
    for j in range(0,len(end_coords)):
        transit_origins.append(i)
        transit_destinations.append(j)
        try:
            response_transit = publicTransitApi.calculate_route(
                departure = list(start_transit_coords[i]),
                arrival = list(end_coords[j]),
                time = '2020-08-04T17:00:00+02')
            #extract trip_time from response
            trip_time = response_transit.as_dict()['Res']['Connections']['Connection'][0]['duration']
            # store trip time in hours
            transit_times.append(isodate.parse_duration(trip_time).seconds/3600)
            #add fare of 3.5
            transit_fares.append(3.5)
        except:
            response_ped = routingApi.matrix(
                start_waypoints=list(start_transit_coords[i:(i+1)]),
                destination_waypoints=list(end_coords[j:(j+1)]),
                departure='2020-08-04T17:00:00+02',
                summary_attributes = ['traveltime','distance'],
                modes=[herepy.RouteMode.fastest, herepy.RouteMode.pedestrian])
            
            trip = response_ped.as_dict()['response']['matrixEntry'][0]
            # if the call fails, it'll return a "status" in the response, to test for that
            if( 'status' in trip.keys()):
                #impute a very large trip time so 
                trip_time = 99999999999
            else: 
                trip_time = trip['summary']['travelTime']
            
            transit_times.append(trip_time/3600)
            #fare is 0 since they're walking
            transit_fares.append(0)
#timestamp
transit_api_end = dt.now()
#store results in tall dataframe
transit_results = pd.DataFrame({"origin":transit_origins,
                                "destination":transit_destinations,
                                "trip_time":transit_times,
                                "transit_fares":transit_fares})
#calculate trip cost using formula in paper
transit_results['trip_costs'] = (transit_results['trip_time']*7.5)+transit_results['transit_fares']

#store times and costs as i x k matrix
transit_time_mat = transit_results.pivot(index='origin',columns='destination',values='trip_time')
transit_cost_mat = transit_results.pivot(index='origin',columns='destination',values='trip_costs')

In [299]:
#combine time matrix and cost matrix
time_mat = pd.concat([car_time_mat,transit_time_mat])
#reset the row numbers
time_mat.reset_index(inplace=True)

cost_mat = pd.concat([car_cost_mat,transit_cost_mat])
cost_mat.reset_index(inplace=True)

#also combine the two household location matrices, which should now be the same number of rows
pittsburgh=pd.concat([residents_car,residents_transit])
pittsburgh.reset_index(inplace=True)


In [300]:
#save csvs of each
time_mat.to_csv('transit_times_100.csv')
cost_mat.to_csv('transit_costs_100.csv')
pittsburgh.to_csv('pittsburgh_100.csv')

In [253]:
pittsburgh['simulated_commute_car']

0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
111    0
112    0
113    0
114    0
115    0
116    0
117    0
118    0
119    0
120    0
121    0
122    0
123    0
124    0
125    0
126    0
127    0
128    0
129    0
130    0
131    0
132    0
133    0
134    0
135    0
136    0
137    0
138    0
139    0
140    0
Name: simulated_commute_car, Length: 141, dtype: int64

In [295]:
car_time_mat

destination,0,1,2,3,4,5,6,7,8,9
origin,,,,,,,,,,
0,0.283889,0.330833,0.357222,0.141389,0.068611,0.363611,0.314722,0.095556,0.270278,0.324167
1,0.506111,0.214167,0.205278,0.346111,0.355556,0.174722,0.536944,0.245000,0.181944,0.305833
2,0.264167,0.308056,0.334444,0.391389,0.276389,0.340833,0.295000,0.288611,0.247500,0.085278
3,0.417500,0.278889,0.305278,0.226667,0.236111,0.311667,0.448333,0.125556,0.218333,0.298611
4,0.387500,0.340556,0.374167,0.418889,0.293056,0.373333,0.419444,0.317778,0.280000,0.133056
5,0.508611,0.376667,0.333611,0.476667,0.414167,0.412500,0.540556,0.375556,0.319167,0.217500
6,0.264167,0.308056,0.334444,0.391389,0.276389,0.340833,0.295000,0.288611,0.247500,0.085278
7,0.416944,0.250833,0.277222,0.248056,0.235000,0.283611,0.447778,0.146944,0.190278,0.272222
8,0.429167,0.268056,0.202222,0.349444,0.326389,0.295000,0.460000,0.248333,0.201667,0.286944


0.7442

25

array([[ 40.4550461, -80.0528284],
       [ 40.4695965, -79.9169844]])

[40.455046100000004, -80.05282840000001]

array([[ 40.4550461, -80.0528284],
       [ 40.4695965, -79.9169844]])

destination,0,1,2,3,4
origin,,,,,
0,0.283889,0.330833,0.357222,0.141389,0.068611
1,0.611667,0.340278,0.298611,0.486111,0.495556
2,0.823611,0.552222,0.510556,0.698056,0.714444
3,0.831111,0.559722,0.518056,0.705556,0.721944
4,0.852500,0.581111,0.539444,0.726944,0.743333


In [70]:
residents_transit

,GEOID,FULL_ADDRESS,MUNICIPALITY,COUNTY,STATE,ZIP_CODE,LATITUDE,LONGITUDE,simulated_income,simulated_commute_car
4,42003562500,2913 ZEPHYR AVE,PITTSBURGH,ALLEGHENY COUNTY,PA,15204.0,40.455046,-80.052828,7533,0
5,42003110600,822 COLLINS AVE,PITTSBURGH,ALLEGHENY COUNTY,PA,15206.0,40.469597,-79.916984,6807,0
7,42003040500,258 1/2 MCKEE PL,PITTSBURGH,ALLEGHENY COUNTY,PA,15213.0,40.438568,-79.958795,55549,0
8,42003480200,1636 ELDERSLEE RD,BALDWIN BOROUGH,ALLEGHENY COUNTY,PA,15227.0,40.360828,-79.959813,94206,0
21,42003563000,1105 VILLAGE RD,PITTSBURGH,ALLEGHENY COUNTY,PA,15205.0,40.449962,-80.075243,49076,0
22,42003110600,6316 JACKSON ST,PITTSBURGH,ALLEGHENY COUNTY,PA,15206.0,40.470284,-79.912368,69871,0
25,42003479000,571 DEWALT DR,BALDWIN TOWNSHIP,ALLEGHENY COUNTY,PA,15234.0,40.378669,-80.015922,951,0
26,42003488600,321 GLENCOE DR,WEST MIFFLIN BOROUGH,ALLEGHENY COUNTY,PA,15122.0,40.349946,-79.913236,3994,0
28,42003562400,226 CHALFONT ST,PITTSBURGH,ALLEGHENY COUNTY,PA,15210.0,40.416420,-80.003113,15152,0
29,42003492800,540 HALCOMB AVE,CLAIRTON,ALLEGHENY COUNTY,PA,15025.0,40.292152,-79.886965,4822,0


In [74]:
start_transit_coords

array([[ 40.4550461, -80.0528284],
       [ 40.4695965, -79.9169844]])

In [225]:
test = isodate.parse_duration(response_transit.as_dict()['Res']['Connections']['Connection'][0]['duration'])
test.seconds

5940

In [129]:
len(start_transit_coords)

166

In [154]:
trip_list[0]

{'startIndex': 0,
 'destinationIndex': 0,
 'summary': {'distance': 8918, 'travelTime': 1022, 'costFactor': 1042}}